In [1]:
from data_utils import get_data_loaders, prepare_train, get_cv_data_loaders
from engine import training_step, validation_step
from utils import print_trainable_parameters
from model import ClsModel, save_model

import pandas as pd
import torch
from torch import nn

In [2]:
df=pd.read_csv("data/train.csv", index_col='id')
X_train, y_train, scaler = prepare_train(df)

In [3]:
for fold, (train_dataloader, val_dataloader) in enumerate(get_cv_data_loaders(X_train, y_train, n_splits=5, batch_size=64)):
    print(f"Fold: {fold}")
    print(f"Train size: {len(train_dataloader)}")
    print(f"Val size: {len(val_dataloader)}")
    print()

Fold: 0
Train size: 3982
Val size: 996

Fold: 1
Train size: 3982
Val size: 996

Fold: 2
Train size: 3982
Val size: 996

Fold: 3
Train size: 3982
Val size: 996

Fold: 4
Train size: 3982
Val size: 996



In [4]:
#instantiating the model
device = 'cuda' if torch.cuda.is_available() else 'cpu'
torch.manual_seed(0)
model  = ClsModel(input_size=21, hidden_size=100,num_classes=1)
adam = torch.optim.Adam(model.parameters(), lr=0.001)
loss_fn = nn.BCELoss()
from torchmetrics import AUROC,Accuracy
auroc = AUROC(num_classes=3,task='binary')
accuracy = Accuracy(task='binary')
train,val = get_data_loaders(X_train, y_train, batch_size=64)

trainable params: 103,301 || all params: 103,301 || trainable%: 100.0


In [ ]:
for i in range(20):
    print(f"Epoch: {i}")
    average_loss, model, optimizer, scheduler = training_step(model, train_dataloader, loss_fn, adam, device)
    print(f"Average loss: {average_loss}")
    print(validation_step(model, val_dataloader, device, rocauc=auroc, accuracy=accuracy))